In [22]:
import pandas as pd
import numpy as np
import msoffcrypto
from io import BytesIO
from pyxlsb import open_workbook
from openpyxl import load_workbook
from openpyxl.styles import Font
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:

# 2025.03.21_Skylark Borrowing Base Report (Protected).xlsb
file_path = "/Users/shivangsingh/Downloads/2025.03.21_Skylark Borrowing Base Report (Protected).xlsb"

sheets_to_import = ["11. Sub BB Schedule", "7. Concentration limits", "5. Advance Rate", "4. Mezz BB Schedule", "3. Snr BB Schedule",
                     "0. Pre-Funding Forecast", "8.B. Employer concentration", "1.B. Data Tape"]


password = "XYZ123"

decrypted_file = BytesIO()
with open(file_path, "rb") as f:
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)  # Load the password
    office_file.decrypt(decrypted_file)

dfs = {}
for sheet in sheets_to_import:
    dfs[sheet] = pd.read_excel(decrypted_file, sheet_name=sheet, engine="pyxlsb")


df_sub_bb = dfs["11. Sub BB Schedule"]
df_conc_lim = dfs["7. Concentration limits"]
df_adv_rate = dfs["5. Advance Rate"]
df_mezz_bb = dfs["4. Mezz BB Schedule"]
df_sen_bb = dfs["3. Snr BB Schedule"]
df_pre_fund =  dfs["0. Pre-Funding Forecast"]
df_emp_conc = dfs["8.B. Employer concentration"]
df_data_tape = dfs["1.B. Data Tape"]

In [ ]:
# 2025.03.21 Skylark Servicer Report (Protected).xlsb
file_path_servicer = "/Users/shivangsingh/Downloads/2025.03.21 Skylark Servicer Report (Protected).xlsb"
password = "ABC123"
sheets_to_import = ["13. Seller Related Events", "12. Asset Related Events", "5. Advance Rate", "3. Snr BB Schedule"]

decrypted_file = BytesIO()
with open(file_path_servicer, "rb") as f:
    office_file = msoffcrypto.OfficeFile(f)
    office_file.load_key(password=password)  # Load the password
    office_file.decrypt(decrypted_file)

dfs = {}
for sheet in sheets_to_import:
    dfs[sheet] = pd.read_excel(decrypted_file, sheet_name=sheet, engine="pyxlsb")

df_seller = dfs["13. Seller Related Events"]
df_asset = dfs["12. Asset Related Events"]
df_advance = dfs["5. Advance Rate"]
df_senior = dfs["3. Snr BB Schedule"]

In [25]:
df_mezz_bb

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Skylark Loans Limited,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Borrowing Base Schedule,NaN,NaN,Mezzanine Borrowing Base Test,pass
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,45735,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
92,NaN,(e),NaN,the Top Ten Employer Partner Excess Concentrat...,NaN,0
93,NaN,(f),NaN,the Unassigned Salary Advance Receivables Exce...,NaN,0
94,NaN,(g),NaN,the Weekly Salary Advance Receivables (Mezzani...,NaN,0
95,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
bb1 = df_mezz_bb.iloc[15, 5] + df_mezz_bb.iloc[20, 5] 
bb2 = df_mezz_bb.iloc[23:26, 5].sum()
bb3= df_mezz_bb.iloc[26:28, 5].sum()
bb4= df_sen_bb.iloc[37, 5] + df_sen_bb.iloc[35, 5] 
bb5 = df_mezz_bb.iloc[34, 5] + df_mezz_bb.iloc[32, 5]
bb6 = bb1+bb2+bb3-(bb4+bb5)

bb7 = df_pre_fund.iloc[47, 4]
bb8 = df_sub_bb.iloc[22, 5]
bb9 = bb7+bb8
bb10 = df_sub_bb.iloc[27, 5]
bb11 = bb10-bb9

print(bb1)
print(bb2)
print(bb3)
print(bb4)
print(bb5)
print(bb6)
print("--------")
print(bb7)
print(bb8)
print(bb9)
print(bb10)
print(bb11)


111282723.55871229
5245393.65
1125255.9
98468000
12160000
7025373.108712301
--------
150000
5730000
5880000
10000000
4120000


## #Raw Data
### Concentration Covenants

In [27]:

# Function to format percentages while handling "-" and NaN values
def format_percent(value):
    if pd.isna(value) or value == "-" or value == "":
        return ""  # Keep it empty if it's NaN, "-", or blank
    return f"{float(value) * 100:.2f}%"

In [28]:
cc1 = df_conc_lim.iloc[12, 5]
cc2 = format_percent(df_conc_lim.iloc[14,5])
cc3 = format_percent(df_emp_conc.iloc[8, 7]) if df_emp_conc.iloc[16, 7] == df_conc_lim.iloc[17,2] else "N/A"
cc4 = format_percent(df_emp_conc.iloc[8, 7]) if df_emp_conc.iloc[16, 7] == df_conc_lim.iloc[18,2] else "N/A"
cc5 = format_percent(df_emp_conc.iloc[13, 7])

numerator_6 = df_data_tape.loc[(df_data_tape["MethodOfPayment"].isin(["Push Payment", "Direct Debit"])) & (df_data_tape["IS ineglible"] == 0), "PrincipalBalanceOutstanding"].sum()
denominator_6 = df_data_tape.loc[df_data_tape["IS ineglible"] == 0, "PrincipalBalanceOutstanding"].sum()
cc6 = format_percent(numerator_6 / denominator_6 if denominator_6 != 0 else 0)

numerator_7 = np.where((df_data_tape["MethodOfPayment"] == "Push Payment") & (df_data_tape["IS ineglible"] == 0), df_data_tape["PrincipalBalanceOutstanding"], 0).sum()
denominator_7 = np.where(df_data_tape["IS ineglible"] == 0, df_data_tape["PrincipalBalanceOutstanding"], 0).sum()
cc7 = format_percent(numerator_7 / denominator_7 if denominator_7 != 0 else 0)

numerator_8 = df_data_tape.loc[(df_data_tape["IsPositiveCounterOffer"] == 1) & (df_data_tape["DelphiCreditBucket"] == "Prime") & (df_data_tape["IS ineglible"] == 0), "PrincipalBalanceOutstanding"].sum()
denominator_8 = df_data_tape.loc[df_data_tape["IS ineglible"] == 0, "PrincipalBalanceOutstanding"].sum()
cc8 = format_percent(numerator_8 / denominator_8 if denominator_8 != 0 else 0)

numerator_9 = df_data_tape.loc[(df_data_tape["PCOwIncreasedMnthlyPmt"] == 1) & (df_data_tape["DelphiCreditBucket"] == "Near Prime") & (df_data_tape["IS ineglible"] == 0), "PrincipalBalanceOutstanding"].sum()
denominator_9 = df_data_tape.loc[df_data_tape["IS ineglible"] == 0, "PrincipalBalanceOutstanding"].sum()
cc9 = format_percent(numerator_9 / denominator_9 if denominator_9 != 0 else 0)

numerator_10 = (df_data_tape["IS ineglible"].eq(0) & df_data_tape["DelphiCreditBucket"].eq("Subprime")) * df_data_tape["PrincipalBalanceOutstanding"]
numerator_sum = numerator_10.sum()
denominator_10 = df_data_tape.loc[df_data_tape["IS ineglible"] == 0, "PrincipalBalanceOutstanding"].sum()
cc10 = format_percent(numerator_sum / denominator_10 if denominator_10 != 0 else 0)

print(cc1)
print(cc2)
print(cc3)
print(cc4)
print(cc5)
print(cc6)
print(cc7)
print(cc8)
print(cc9)
print(cc10)

35.102517825312866
22.63%
8.08%
N/A
28.78%
6.79%
0.40%
2.39%
3.51%
2.46%


### Financial/Performance Covenants

In [29]:
df_row_27 = pd.to_numeric(df_seller.loc[27], errors="coerce").dropna()
df_row_27 = pd.to_datetime(df_row_27, origin='1899-12-30', unit='D')

last_date_column = df_row_27.last_valid_index()

last_valid_col_number = df_seller.columns.get_loc(last_date_column)
last_valid_col_number

27

In [30]:
fc_test = df_seller.iloc[52, -1]
fc_test

np.float64(nan)

In [31]:
fc1 = df_seller.iloc[52, last_valid_col_number]
fc2 = df_seller.iloc[64, last_valid_col_number]
print(fc1)
print(fc2)

23572221.57
8904762.379999999


In [32]:
fc3 = format_percent(df_advance.iloc[132, 4:7].mean())
fc4 = format_percent(df_advance.iloc[122, 4:7].mean())
fc5 = format_percent(df_advance.iloc[82, 5])
fc6 = df_asset.iloc[15,7]
print(fc3)
print(fc4)
print(fc5)
print(fc6)

1.50%
0.64%
8.38%
720345.0


In [33]:
sc1 = format_percent(df_adv_rate.iloc[86,5])
sc2 = df_senior.iloc[37,5]
print(sc1)
print(sc2)

0.64%
96046000


In [34]:
data = {
    "Metric": [
        "Raw Data",
        "",
        "Compliance Certificate", "Quarter", "Covenants", "Concentration Covenants",
        "Average Weighted Remaing Term",
        "Minimum WA APR",
        "Single Employer (if <100m)",
        "Single Employer (if >100m)",
        "Top 5 Employer partenr",
        "Direct Debit/Push Payments",
        "Push Payments only",
        "Adjusted Loan Terms Prime Borrowers only",
        "Adjusted Loan Terms Near Prime Borrowers only",
        "Sub-Prime Borrowers",
        "",
        "Financial/Performance Covenants",
        "Tangible net worth",
        "Liquidity and Min Cash",
        "Deliquency Rate",
        "Default Rate",
        "Payment Not By Salary Deduction",
        "Balance of Interest Reserve Account",
        "Senior Covenants",
        "Default Rate",
        "Senior Drawn ",
        "",
        "Borrowing Base",
        "",
        "Subordinate Borrowing Base", "",
        "the Eligible Receivables Balance as at the immediately preceding Cut-Off Date; plus ",
        "the sum of:",
        "the aggregate sum of the balance standing to the credit of all of the Issuer Accounts",
        "the aggregate sum of the balance standing to the credit of all of the Sandpit Accounts and Disbursement Accounts",
        "the principal amount outstanding of the Senior Loan on such date; and ",
        "the principal amount outstanding of the Mezzanine Note on such date",
        "Subordinated Borrowing Base = (a) + (b) - (c)",
        "Subordinate funding request for valuation on reporting date",
        "Current balance outstanding on subordinate funding line (before current funding request)",
        "",
        "Total commitment drawn to date (inlcuding current advance request)",
        "",
        "Total agreed facility commitment ",
        "",
        "Total funds available to be drawn (after current advance request)"
    ],
    "Value": [
        "",
        "",
        "",  "", "", "",
        cc1, cc2, cc3, cc4, cc5, cc6, cc7, cc8, cc9, cc10,
        "",
        "",
        fc1, fc2, fc3, fc4, fc5, fc6,
        "",
        sc1, sc2,
        "",
        "", "", "", "",
        bb1, "", bb2, bb3, bb4, bb5, bb6, bb7, bb8, 
        "", bb9, "", bb10, "", bb11
    ]
}
df_final = pd.DataFrame(data)
df_final

,Metric,Value
0,Raw Data,
1,,
2,Compliance Certificate,
3,Quarter,
4,Covenants,
5,Concentration Covenants,
6,Average Weighted Remaing Term,35.102518
7,Minimum WA APR,22.63%
8,Single Employer (if <100m),8.08%
9,Single Employer (if >100m),N/A


In [35]:
print("Length of 'Metric' list:", len(data["Metric"]))
print("Length of 'Value' list:", len(data["Value"]))


Length of 'Metric' list: 47
Length of 'Value' list: 47


In [36]:

file_name = "salaryfinance_monitoring_sheet.xlsx"
df_final.to_excel(file_name, index=False, header = False)
wb = load_workbook(file_name)
ws = wb.active

for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=2):
    metric_cell, value_cell = row
    if value_cell.value in ["", None]: 
        metric_cell.font = Font(bold=True)

wb.save(file_name)
print(f"Formatted Excel file saved as {file_name}")

Formatted Excel file saved as salaryfinance_monitoring_sheet.xlsx


In [21]:

date_str = file_path_servicer.split("/")[-1].split(" ")[0] 

date_obj = datetime.strptime(date_str, "%Y.%m.%d")

formatted_date = date_obj.strftime("01/%m/%Y")

previous_month_date = date_obj - relativedelta(months=1)
formatted_prev_date = previous_month_date.strftime("01/%m/%Y")

print("Extracted Date:", formatted_date)
print("Previous Month Date:", formatted_prev_date)


Extracted Date: 01/03/2025
Previous Month Date: 01/02/2025
